In [32]:
import requests
from bs4 import BeautifulSoup
import os

dir_paginas = 'paginas'

In [33]:
# baixar página
# buscar os produtos
# para cada produto
    # baixar a página
    # encontrar a data, o preço e o local do treinamento
    # salvar a informação
# imprimir as informações

In [34]:
#%% Change working directory from the workspace root to the ipynb file location. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'Aula2/Scraper'))
	print(os.getcwd())
except:
	pass


In [35]:
def buscarPagina(url):
    """
    Faz o download de 1 página passada como parâmetro
    
    Args:
        url: url a ser buscada
    
    Returns:
        texto da página
        
    Examples:
        buscarPagina('https://escolaconquer.com.br/xxxxx/')
    """
    # gera o nome completo do arquivo com o caminho da máquina
    nome = url.replace('/', '#').replace('.', '#').replace(':', '#')
    arquivo = os.path.join(dir_paginas, nome+'.htm')
    print(arquivo)

    # abre o arquivo e lê as informações
    if os.path.exists(arquivo):
        f = open(arquivo, 'r')
        data = f.read()
        
        print("reading {}".format(arquivo))
        return data
    
def encontrarUrls(texto, url):
    """
    Encontra uma URL no texto.
    Args:
        texto: texto da página
        url: url a ser buscada
    Returns:
        lista de urls encontradas na página que contenham a url passada
        
    Examples:
        encontrarUrls(html_resp.text, 'https://escolaconquer.com.br/xxxxx/')
    """
    # cria um objeto beautifulsoup para poder procurar como html
    soup = BeautifulSoup(texto, 'html.parser')
    
    # procura todos os links que possuem o url mencionado
    links = soup.find_all('a', href=lambda href: href and url in href)
    
    # retorna uma lista com todos os links
    ret = [link.attrs['href'] for link in links]
    return ret


    

In [36]:
infos = {}

links_produtos = [
    'https://escolaconquer.com.br/produto/grow-your-business/cwb/',
    'https://escolaconquer.com.br/produto/alta-performance-poa/',
    'https://escolaconquer.com.br/produto/alta-performance-rj/',
    'https://escolaconquer.com.br/produto/alta-performance-bh/',
    'https://escolaconquer.com.br/produto/alta-performance-sp/',
]
for lp in links_produtos:
    # busca a página
    texto_produto = buscarPagina(lp)
    infos[lp] = texto_produto


paginas/https###escolaconquer#com#br#produto#grow-your-business#cwb#.htm
reading paginas/https###escolaconquer#com#br#produto#grow-your-business#cwb#.htm
paginas/https###escolaconquer#com#br#produto#alta-performance-poa#.htm
reading paginas/https###escolaconquer#com#br#produto#alta-performance-poa#.htm
paginas/https###escolaconquer#com#br#produto#alta-performance-rj#.htm
reading paginas/https###escolaconquer#com#br#produto#alta-performance-rj#.htm
paginas/https###escolaconquer#com#br#produto#alta-performance-bh#.htm
reading paginas/https###escolaconquer#com#br#produto#alta-performance-bh#.htm
paginas/https###escolaconquer#com#br#produto#alta-performance-sp#.htm
reading paginas/https###escolaconquer#com#br#produto#alta-performance-sp#.htm


In [37]:
# busca a página e os produtos
# texto_pagina = buscarPagina('http://www.escolaconquer.com.br')
# links_produtos = encontrarUrls(texto_pagina, 'https://escolaconquer.com.br/produto/')

# para cada um dos produtos
for lp in links_produtos:
    
    # imprime algo para saber que está funcionando
    print('###########')
    print("Processando página {}".format(lp))
    
    # busca a página
    texto_produto = buscarPagina(lp)
    soup = BeautifulSoup(texto_produto, 'html.parser')
    
    # pelo URL, identifica o nome do produto e o local
    url_dividido = lp.split('/')
    produto, cidade = url_dividido[4:6]
    produto = produto.split("-")        
    
    # verifica o tipo da página
    if len(url_dividido) == 7:
        # tipo de página novo
        produto = " ".join(produto).capitalize()
        
        # coleta a informação de preco
        preco = soup.select('.investimento-preco')[0].text
        
        # coleta as informacoes dos horarios
        a = soup.select('#turmas-mob')[0]
        for b in a.select('div.wpb_content_element'):
            # este código é bem específico para esta página, já que não há ids
            if(b.h3):
                local = b.h3.text.split("::")[0]
            else:
                p_list = b.div.select('p')
                if(len(p_list))==6:
                    # tira as informações da tabela
                    data = p_list[0].text
                    freq = p_list[1].text
                    hora = p_list[3].text

        print('------')
        print(local, data, freq, hora)
        
    else:
        # tipo de página antigo        
        cidade = produto[-1]
        produto = produto[:-1]
        produto = " ".join(produto).capitalize()

        # coleta a informação de preço
        preco = soup.select('.price-installment')[0].text
        
        # verifica é uma página com mais de um local (ex:SP)
        locais = soup.select('[id*="childrenContent"]')
        
        # para cada um dos cards de disponibilidade, pega as informações
        if len(locais)==0:
            cards = soup.find_all('div', {"class": "card-product-offers"})
            for card in cards:
                turma = card.select('.card-product-offers-title')[0].text 
                titulos = card.select('.card-product-offers-item-title')
                data = titulos[0].text.strip()
                freq = titulos[1].text.strip()
                hora = titulos[2].text.strip()

                print('------')
                print(cidade, data, freq, hora)
        else:
            # existe mais de um local. Acha o card deles individualmente
            for local in locais:
                # pega o titulo do local
                nome_local = local.h4.strong.text
                
                # pega as outras informações
                cards = local.find_all('div', {"class": "card-product-offers"})
                for card in cards:
                    turma = card.select('.card-product-offers-title')[0].text 
                    titulos = card.select('.card-product-offers-item-title')
                    data = titulos[0].text.strip()
                    freq = titulos[1].text.strip()
                    hora = titulos[2].text.strip()

                    print('------')
                    print(nome_local, data, freq, hora)

###########
Processando página https://escolaconquer.com.br/produto/grow-your-business/cwb/
paginas/https###escolaconquer#com#br#produto#grow-your-business#cwb#.htm
reading paginas/https###escolaconquer#com#br#produto#grow-your-business#cwb#.htm
------
curitiba  26/mar a 02/jul Encontros presenciais 1x por semana, todas as 3ª feiras 18h30 às 22h30
###########
Processando página https://escolaconquer.com.br/produto/alta-performance-poa/
paginas/https###escolaconquer#com#br#produto#alta-performance-poa#.htm
reading paginas/https###escolaconquer#com#br#produto#alta-performance-poa#.htm
------
poa 27/03 a 15/05 aula 1x por semana 19h às 22h15
------
poa 30/03 a 11/05 aula 1x por semana 09h30 às 12h45
------
poa 30/04 a 04/06 aula 1x por semana 19h às 22h15
###########
Processando página https://escolaconquer.com.br/produto/alta-performance-rj/
paginas/https###escolaconquer#com#br#produto#alta-performance-rj#.htm
reading paginas/https###escolaconquer#com#br#produto#alta-performance-rj#.htm


In [41]:
#!/usr/bin/python
# -*- coding: 'latin-1' -*-
#
# ConquerX: Introducao a programacao - Python
# Programa que fará um scraping da página da Conquer, buscando os Produtos
#
import logging

# classes criadas por nós
from utils import paginasConquer as pc
from cursoConquer import CursoConquer
import pandas as pd

dir_paginas = 'paginas'

logging.basicConfig()
logging.getLogger('root').setLevel(logging.INFO)

if __name__ == "__main__":
    # busca a página e os produtos
    # texto_pagina = buscarPagina('https://escolaconquer.com.br')
    # links_produtos = encontrarUrls(texto_pagina, 'https://escolaconquer.com.br/produto/')

    # usaremos este para não gerar sobrecarga nos servidores
    links_produtos = [
    'https://escolaconquer.com.br/produto/grow-your-business/cwb/',
    'https://escolaconquer.com.br/produto/alta-performance-poa/',
    'https://escolaconquer.com.br/produto/alta-performance-rj/',
    'https://escolaconquer.com.br/produto/alta-performance-bh/',
    'https://escolaconquer.com.br/produto/alta-performance-sp/'
    ]

    # para cada um dos produtos
    lista_produtos = []
    for lp in links_produtos:
        
        # imprime algo para saber que está funcionando
        print('###########')
        print("Processando página {}".format(lp))
        
        # pelo URL, identifica o nome do produto e o local
        url_dividido = lp.split('/')
        produto, cidade = url_dividido[4:6]
        produto = produto.split("-")  
        
        # busca a página
        texto_produto = pc.buscarPagina(lp)

        # verifica o tipo da página
        if len(url_dividido) == 7:
            # tipo de página novo
            produto = " ".join(produto).capitalize()
            
            # busca as informacoes
            local, data, freq, hora, preco = pc.infoPaginaNova(texto_produto)[0]
            curso = CursoConquer(produto, cidade, local, data, freq, preco)
            lista_produtos.append(curso)
            
        else:
            # tipo de página antigo        
            cidade = produto[-1]
            produto = produto[:-1]
            produto = " ".join(produto).capitalize()

            # busca as informações
            lista_cursos = pc.infoPaginaAntiga(texto_produto)
            for info_curso in lista_cursos:
                local, data, freq, hora, preco = info_curso
                curso = CursoConquer(produto, cidade, local, data, freq, preco)
                lista_produtos.append(curso)

    # imprime como uma tabela


###########
Processando página https://escolaconquer.com.br/produto/grow-your-business/cwb/
###########
Processando página https://escolaconquer.com.br/produto/alta-performance-poa/
###########
Processando página https://escolaconquer.com.br/produto/alta-performance-rj/
###########
Processando página https://escolaconquer.com.br/produto/alta-performance-bh/
###########
Processando página https://escolaconquer.com.br/produto/alta-performance-sp/


In [44]:
pd.DataFrame([ dict(prod) for prod in lista_produtos])

,cidade,data,freq,local,nome,preco
0,cwb,26/mar a 02/jul,"Encontros presenciais 1x por semana, todas as ...",curitiba,Grow your business,09xR$630
1,poa,27/03 a 15/05,aula 1x por semana,,Alta performance,Por 4x de R$ 390
2,poa,30/03 a 11/05,aula 1x por semana,,Alta performance,Por 4x de R$ 390
3,poa,30/04 a 04/06,aula 1x por semana,,Alta performance,Por 4x de R$ 390
4,rj,28/03 a 09/05,aula 1x por semana,,Alta performance,Por 4x de R$ 490
5,rj,30/03 a 11/05,aula 1x por semana,,Alta performance,Por 4x de R$ 490
6,bh,29/04 a 03/06,aula 1x por semana,,Alta performance,Por 4x de R$ 390
7,bh,04/05 a 08/06,aula 1x por semana,,Alta performance,Por 4x de R$ 390
8,sp,30/04 a 04/06,aula 1x por semana,Av. Paulista,Alta performance,Por 4x de R$ 490
9,sp,27/03 a 08/05,aula 1x por semana,Vila Olímpia,Alta performance,Por 4x de R$ 490


In [3]:
from utils import paginasConquer as pc
import requests
import os, sys
import logging
import time
import traceback
from bs4 import BeautifulSoup 

In [12]:
html = pc.buscarPagina('https://escolaconquer.com.br/produto/aprenda-a-programar-em-uma-semana/cwb/')


# transforma a página em soup para melhorar a busca
soup = BeautifulSoup(html, 'html.parser')

# coleta a informação de preco
preco = soup.select('.investimento-preco')[0].text

# coleta as informacoes dos horarios
turmas = soup.select('#turmas-mob')[0]
for info in turmas.select('div.wpb_content_element'):
    # este código é bem específico para esta página, já que não há ids
    if(info.h3):
        local = info.h3.text.split("::")[0]
    else:
        p_list = info.div.select('p')
        print(len(p_list))
        if len(p_list) in [5,6]:
            # tira as informações da tabela
            data = p_list[0].text.replace("\n", " ")
            freq = p_list[1].text.replace("\n", " ")
            hora = p_list[3].text.replace("\n", " ")
        if(len(p_list)==5):
            hora = None
            
[(local, data, freq, hora, preco)]

0
0
5
1


[('curitiba ',
  'Data: 22/04 a 27/04',
  'Horários: Segunda a quinta: 19:00 às 22:15 Sábado: 09:00 às 16:30',
  None,
  '03xR$330')]

In [14]:

nomes = [
     "Alice,Miguel
    , Sophia,Arthur
    ,Helena,Bernardo
    ,Valentina,Heitor
    ,Laura,Davi
    ,Isabella,Lorenzo
    ,Manuela,Théo
    ,Júlia,Pedro
    ,Heloísa,Gabriel
    ,Luiza,Enzo
    ,Maria Luiza,Matheus
    ,Lorena,Lucas
    ,Lívia,Benjamin
    ,Giovanna,Nicolas
    ,Maria Eduarda,Guilherme
    ,Beatriz,Rafael
    ,Maria Clara,Joaquim
    ,Cecília,Samuel
    ,Eloá,Enzo Gabriel
    ,Lara,João Miguel
    ,Maria Júlia,Henrique
    ,Isadora,Gustavo
]

SyntaxError: invalid syntax (<ipython-input-14-8ffe8d607ac3>, line 13)

In [21]:
nomes = """Alice
    Miguel
    Sophia
    Arthur
    Helena
    Bernardo
    Valentina
    Heitor
    Laura
    Davi
    Isabella
    Lorenzo
    Manuela
    Théo
    Júlia
    Pedro
    Heloísa
    Gabriel
    Luiza
    Enzo
    Maria Luiza
    Matheus
    Lorena
    Lucas
    Lívia
    Benjamin
    Giovanna
    Nicolas
    Maria Eduarda
    Guilherme
    Beatriz
    Rafael
    Maria Clara
    Joaquim
    Cecília
    Samuel
    Eloá
    Enzo Gabriel
    Lara
    João Miguel
    Maria Júlia
    Henrique
    Isadora
    Gustavo"""
nomes = nomes.split("\n    ")

In [22]:
sobrenomes = 

['Alice',
 'Miguel',
 'Sophia',
 'Arthur',
 'Helena',
 'Bernardo',
 'Valentina',
 'Heitor',
 'Laura',
 'Davi',
 'Isabella',
 'Lorenzo',
 'Manuela',
 'Théo',
 'Júlia',
 'Pedro',
 'Heloísa',
 'Gabriel',
 'Luiza',
 'Enzo',
 'Maria Luiza',
 'Matheus',
 'Lorena',
 'Lucas',
 'Lívia',
 'Benjamin',
 'Giovanna',
 'Nicolas',
 'Maria Eduarda',
 'Guilherme',
 'Beatriz',
 'Rafael',
 'Maria Clara',
 'Joaquim',
 'Cecília',
 'Samuel',
 'Eloá',
 'Enzo Gabriel',
 'Lara',
 'João Miguel',
 'Maria Júlia',
 'Henrique',
 'Isadora',
 'Gustavo']

In [23]:
sobrenomes = """Agostinho
Aguiar
Albuquerque
Alegria
Alencastro
Almada
Alves
Alvim
Amorim
Andrade
Antunes
Aparício
Apolinário
Araújo
Arruda
Assis
Assunção
Ávila
Azambuja
Baptista
Barreto
Barros
Beira-Mar
Belchior
Belém
Bernardes
Bittencourt
Boaventura
Bonfim
Botelho
Brites
Brito
Caetano
Calixto
Camacho
Camilo
Capelo
Castro
Cavalcante
Chaves
Conceição
Corte Real
Cortês
Coutinho
Crespo
Cunha
Curado
Custódio
Córdoba
Damásio
Dantas
Dias
Dinís
Domingues
Dorneles
dos Reis
Drumond
D’Ávila
Escobar
Espinosa
Esteves
Evangelista
Farias
Ferrari
Figueiredo
Figueiroa
Flores
Fogaça
Freitas
Frutuoso
Furtado
Félix
Galvão
Garcia
Gaspar
Gentil
Geraldes
Gil
Godinho
Gomes
Gonzaga
Goulart
Gouveia
Guedes
Guimarães
Guterres
Góis
Hernandes
Hilário
Hipólito
Ibrahim
Ilha
Infante
Jaques
Jesus
Jordão
Lacerda
Lancastre
Leiria
Lessa
Machado
Maciel
Magalhães
Maia
Maldonado
Marinho
Marques
Martins
Medeiros
Meireles
Mello
Mendes
Menezes
Mesquita
Modesto
Monteiro
Morais
Moreira
Morgado
Moura
Muniz
Neves
Nogueira
Novais
Nóbrega
Ornélas
Oliveira
Ourique
Pacheco
Padilha
Paiva
Paraíso
Paris
Peixoto
Peralta
Peres
Pilar
Pimenta
Pinheiro
Portela
Quaresma
Quarteira
Queiroz
Ramires
Ramos
Rebelo
Resende
Ribeiro
Salazar
Sales
Salgado
Salgueiro
Sampaio
Sanches
Santana
Siqueira
Soares
Subtil
Tavares
Taveira
Teixeira
Teles
Torres
Trindade
Varela
Vargas
Vasconcelos
Vasques
Veiga
Veloso
Vidal
Vieira
Vilela
Xavier
Ximenes
Xisco
Zagalo
Zanette
Zaganelli"""

In [24]:
sobrenomes =  sobrenomes.split("\n")

In [25]:
sobrenomes

['Agostinho',
 'Aguiar',
 'Albuquerque',
 'Alegria',
 'Alencastro',
 'Almada',
 'Alves',
 'Alvim',
 'Amorim',
 'Andrade',
 'Antunes',
 'Aparício',
 'Apolinário',
 'Araújo',
 'Arruda',
 'Assis',
 'Assunção',
 'Ávila',
 'Azambuja',
 'Baptista',
 'Barreto',
 'Barros',
 'Beira-Mar',
 'Belchior',
 'Belém',
 'Bernardes',
 'Bittencourt',
 'Boaventura',
 'Bonfim',
 'Botelho',
 'Brites',
 'Brito',
 'Caetano',
 'Calixto',
 'Camacho',
 'Camilo',
 'Capelo',
 'Castro',
 'Cavalcante',
 'Chaves',
 'Conceição',
 'Corte Real',
 'Cortês',
 'Coutinho',
 'Crespo',
 'Cunha',
 'Curado',
 'Custódio',
 'Córdoba',
 'Damásio',
 'Dantas',
 'Dias',
 'Dinís',
 'Domingues',
 'Dorneles',
 'dos Reis',
 'Drumond',
 'D’Ávila',
 'Escobar',
 'Espinosa',
 'Esteves',
 'Evangelista',
 'Farias',
 'Ferrari',
 'Figueiredo',
 'Figueiroa',
 'Flores',
 'Fogaça',
 'Freitas',
 'Frutuoso',
 'Furtado',
 'Félix',
 'Galvão',
 'Garcia',
 'Gaspar',
 'Gentil',
 'Geraldes',
 'Gil',
 'Godinho',
 'Gomes',
 'Gonzaga',
 'Goulart',
 'Gouveia',


In [35]:
lista_nome_sobrenome = [
        "{} - {} {}".format(id+1000000, nome, sobrenome)
        for id, (nome, sobrenome) in enumerate(zip(random.choices(nomes, k=20), 
                                   random.choices(sobrenomes, k=20)))
    ]

In [36]:
lista_nome_sobrenome

['1000000 - Guilherme Monteiro',
 '1000001 - Valentina Hernandes',
 '1000002 - Lívia Xavier',
 '1000003 - Lucas Vieira',
 '1000004 - Isabella Ourique',
 '1000005 - Miguel Monteiro',
 '1000006 - Gabriel Paris',
 '1000007 - Cecília Assis',
 '1000008 - Benjamin Ibrahim',
 '1000009 - Manuela Pilar',
 '1000010 - Henrique Mendes',
 '1000011 - João Miguel Salazar',
 '1000012 - Matheus Cavalcante',
 '1000013 - Maria Luiza Maia',
 '1000014 - Maria Clara Ibrahim',
 '1000015 - Arthur Barros',
 '1000016 - Heitor Boaventura',
 '1000017 - Maria Eduarda Mesquita',
 '1000018 - Nicolas Apolinário',
 '1000019 - João Miguel Antunes']